In [ ]:
# 2D CNN 批量版
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# 参数设置
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                  # 拼接时每组多少条信号
test_ratio = 0.25        # 测试集比例

# 调用函数
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_all_mix_random(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# ---------- PyTorch & 训练配置 ----------
import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt  # 保留，若需扩展
import math

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据增强 / 信道设置（目前默认关闭）
USE_LOG = True
FS = 20e6
FC = 2.4e9
SNR_DB = 20
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True

# 训练参数
BATCH_SIZE = 256   # 更大 batch 更稳定（你数据很多）
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 1e-4
N_SPLITS = 5
PATIENCE = 8
MIN_DELTA = 0.1   # early stopping 的最小进步（%）

SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

# ---------- 可选：AWGN / Doppler（保留） ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 数据预处理：针对你的交错样本，用 per-sample 标准化，不做 FFT ----------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: np.array [N, L, 2] (I,Q)
    输出 torch.tensor [N, L, 2] (float32)，每个样本做 zero-mean unit-std 标准化（按样本）
    说明：不做 FFT，因为样本不是连续时间序列（已交错）。
    """
    data = data_real_imag.astype(np.float32).copy()
    N, L, C = data.shape
    out = np.empty_like(data, dtype=np.float32)
    for i in range(N):
        iq = data[i]  # (L,2)
        # 可选：先把 (I,Q) 组合成 complex 做 channel 级增强 (保留)
        if add_noise:
            # 以 complex 形式添加噪声
            sigc = iq[...,0] + 1j * iq[...,1]
            sigc = add_complex_awgn(sigc, snr_db)
            if add_doppler:
                sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)
            iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        # per-sample 标准化（对两个通道一起）
        mu = iq.mean(axis=(0,))    # shape (2,)
        sigma = iq.std(axis=(0,))  # shape (2,)
        # 防止除零
        sigma[sigma < 1e-8] = 1.0
        iq_norm = (iq - mu) / sigma
        out[i] = iq_norm

    return torch.tensor(out, dtype=torch.float32)

# ---------- 新的 CNN：IQPointCloudCNN（把每个样本当作 [1,240,2] 图像） ----------
class IQPointCloudCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super().__init__()
        # 输入: [B, 1, L, 2]
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(7,2), stride=1, padding=(3,0), bias=False),  # -> [B,32,L,1]
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,1)),  # -> [B,32,L/2,1]

            nn.Conv2d(32, 64, kernel_size=(5,1), padding=(2,0), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,1)),  # -> [B,64,L/4,1]

            nn.Conv2d(64, 128, kernel_size=(3,1), padding=(1,0), bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,1)),  # -> [B,128,L/8,1]

            nn.Conv2d(128, 256, kernel_size=(3,1), padding=(1,0), bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,1)),  # -> [B,256,L/16,1]
        )
        # 计算 flatten 大小：L 为 240 (你的设置)，L//16 = 15
        # 256 * 15 * 1 = 3840
        self.flatten_dim = 256 * (240 // 16) * 1
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: [B, L, 2]
        if x.dim() == 3:
            x = x.unsqueeze(1)  # -> [B,1,L,2]
        x = self.features(x)   # -> [B, 256, L/16, 1]
        out = self.classifier(x)
        return out

# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()

# ---------- K-Fold 训练主函数（使用 IQPointCloudCNN） ----------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes, device=DEVICE,
                           batch_size=BATCH_SIZE, epochs=EPOCHS, lr=LR, weight_decay=WEIGHT_DECAY,
                           n_splits=N_SPLITS, patience=PATIENCE, min_delta=MIN_DELTA,
                           script_name="wisig_LED_cross"):
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}_fd{fd}_classes_{num_classes}_CNN"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # test loader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # full dataset for K-Fold
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model = IQPointCloudCNN(num_classes=num_classes, dropout=0.3).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())
            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                  f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                        f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping.")
                    break

            scheduler.step()

        # restore best
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')

        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')

        # test eval
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

        # 保存曲线 & 模型
        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ---------- 运行前的准备（预处理并转换为张量） ----------
# X_train, X_test 目前 shape: [N, 240, 2] (numpy)
# 我们对每个样本做 per-sample 标准化（zero-mean unit-std）
print("Preprocessing (per-sample normalization)...")
# ---------- 转为 torch.tensor 并做 per-sample 标准化 ----------

X_train_torch = preprocess_for_pointcloud_cnn(X_train, add_noise=False)
X_test_torch  = preprocess_for_pointcloud_cnn(X_test, add_noise=False)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch  = torch.tensor(y_test, dtype=torch.long)

num_classes = len(torch.unique(y_train_torch))
print("Prepared tensors:", X_train_torch.shape, X_test_torch.shape, "num_classes=", num_classes)

# ---------- K-Fold 训练 ----------
# train_kfold_pointcloud 内部会把训练集再划分为训练+验证集
#save_folder = train_kfold_pointcloud(
#    X_train_torch, 
#    y_train_torch,
#    X_test_torch, 
#    y_test_torch,
#    num_classes=num_classes
#)

#print("Done. Results in:", save_folder)

###########################################################
# === 定义一个统一入口函数，用于跑一次 SNR 实验 ===
###########################################################
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE

    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    # 设置当前实验 SNR
    SNR_DB = snr_db
    ADD_NOISE = True   # 打开 AWGN
    
    # --- 重新预处理数据（会调用 add_noise=True） ---
    X_train_torch_snr = preprocess_for_pointcloud_cnn(
        X_train, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )
    X_test_torch_snr = preprocess_for_pointcloud_cnn(
        X_test, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )

    y_train_torch_snr = torch.tensor(y_train, dtype=torch.long)
    y_test_torch_snr  = torch.tensor(y_test,  dtype=torch.long)

    num_classes = len(torch.unique(y_train_torch_snr))

    # --- 调用你的 K-Fold 训练 ---
    save_folder = train_kfold_pointcloud(
        X_train_torch_snr,
        y_train_torch_snr,
        X_test_torch_snr,
        y_test_torch_snr,
        num_classes=num_classes,
        script_name=f"SNR_{snr_db}dB"
    )

    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder


###########################################################
# === 批量执行 SNR Sweep: 20, 10, 0, -10, ... -40 dB ===
###########################################################
snr_list = list(range(20, -41, -5))
# [20, 10, 0, -10, -20, -30, -40]

all_results = {}

for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")



In [ ]:
# ResNet 批量版
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# 参数设置
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                   # 拼接时每组多少条信号
test_ratio = 0.25       # 测试集比例

# 调用函数
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_all_mix_random(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# ---------- PyTorch & 训练配置 ----------
import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt  # 保留，若需扩展
import math

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据增强 / 信道设置（目前默认关闭）
USE_LOG = True
FS = 20e6
FC = 2.4e9
SNR_DB = 20
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True

# 训练参数
BATCH_SIZE = 256   # 更大 batch 更稳定（你数据很多）
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 1e-4
N_SPLITS = 5
PATIENCE = 8
MIN_DELTA = 0.1   # early stopping 的最小进步（%）

SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

# ---------- 可选：AWGN / Doppler（保留） ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 数据预处理：针对你的交错样本，用 per-sample 标准化，不做 FFT ----------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: np.array [N, L, 2] (I,Q)
    输出 torch.tensor [N, L, 2] (float32)，每个样本做 zero-mean unit-std 标准化（按样本）
    说明：不做 FFT，因为样本不是连续时间序列（已交错）。
    """
    data = data_real_imag.astype(np.float32).copy()
    N, L, C = data.shape
    out = np.empty_like(data, dtype=np.float32)
    for i in range(N):
        iq = data[i]  # (L,2)
        # 可选：先把 (I,Q) 组合成 complex 做 channel 级增强 (保留)
        if add_noise:
            # 以 complex 形式添加噪声
            sigc = iq[...,0] + 1j * iq[...,1]
            sigc = add_complex_awgn(sigc, snr_db)
            if add_doppler:
                sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)
            iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        # per-sample 标准化（对两个通道一起）
        mu = iq.mean(axis=(0,))    # shape (2,)
        sigma = iq.std(axis=(0,))  # shape (2,)
        # 防止除零
        sigma[sigma < 1e-8] = 1.0
        iq_norm = (iq - mu) / sigma
        out[i] = iq_norm

    return torch.tensor(out, dtype=torch.float32)

# ---------- 新的 CNN：IQPointCloudCNN（把每个样本当作 [1,240,2] 图像） ----------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out


class RF1DCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super().__init__()

        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # 240 → 120

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # 120 → 60

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # 60 → 30

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # 30 → 15

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 15, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, 240, 2)
        x = x.permute(0, 2, 1)  # → (B, 2, 240)

        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)

        return self.fc(x)


# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()

# ---------- K-Fold 训练主函数（使用 IQPointCloudCNN） ----------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes, device=DEVICE,
                           batch_size=BATCH_SIZE, epochs=EPOCHS, lr=LR, weight_decay=WEIGHT_DECAY,
                           n_splits=N_SPLITS, patience=PATIENCE, min_delta=MIN_DELTA,
                           script_name="wisig_LED_cross"):
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}_fd{fd}_classes_{num_classes}_CNN"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # test loader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # full dataset for K-Fold
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model = RF1DCNN(num_classes=num_classes, dropout=0.3).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())
            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                  f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                        f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping.")
                    break

            scheduler.step()

        # restore best
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')

        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')

        # test eval
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

        # 保存曲线 & 模型
        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ---------- 运行前的准备（预处理并转换为张量） ----------
# X_train, X_test 目前 shape: [N, 240, 2] (numpy)
# 我们对每个样本做 per-sample 标准化（zero-mean unit-std）
print("Preprocessing (per-sample normalization)...")
# ---------- 转为 torch.tensor 并做 per-sample 标准化 ----------

X_train_torch = preprocess_for_pointcloud_cnn(X_train, add_noise=False)
X_test_torch  = preprocess_for_pointcloud_cnn(X_test, add_noise=False)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch  = torch.tensor(y_test, dtype=torch.long)

num_classes = len(torch.unique(y_train_torch))
print("Prepared tensors:", X_train_torch.shape, X_test_torch.shape, "num_classes=", num_classes)

# ---------- K-Fold 训练 ----------
# train_kfold_pointcloud 内部会把训练集再划分为训练+验证集
#save_folder = train_kfold_pointcloud(
#    X_train_torch, 
#    y_train_torch,
#    X_test_torch, 
#    y_test_torch,
#    num_classes=num_classes
#)

#print("Done. Results in:", save_folder)

###########################################################
# === 定义一个统一入口函数，用于跑一次 SNR 实验 ===
###########################################################
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE

    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    # 设置当前实验 SNR
    SNR_DB = snr_db
    ADD_NOISE = True   # 打开 AWGN
    
    # --- 重新预处理数据（会调用 add_noise=True） ---
    X_train_torch_snr = preprocess_for_pointcloud_cnn(
        X_train, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )
    X_test_torch_snr = preprocess_for_pointcloud_cnn(
        X_test, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )

    y_train_torch_snr = torch.tensor(y_train, dtype=torch.long)
    y_test_torch_snr  = torch.tensor(y_test,  dtype=torch.long)

    num_classes = len(torch.unique(y_train_torch_snr))

    # --- 调用你的 K-Fold 训练 ---
    save_folder = train_kfold_pointcloud(
        X_train_torch_snr,
        y_train_torch_snr,
        X_test_torch_snr,
        y_test_torch_snr,
        num_classes=num_classes,
        script_name=f"SNR_{snr_db}dB"
    )

    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder


###########################################################
# === 批量执行 SNR Sweep: 20, 10, 0, -10, ... -40 dB ===
###########################################################
snr_list = list(range(20, -41, -5))
# [20, 10, 0, -10, -20, -30, -40]

all_results = {}

for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")



In [ ]:
# ResNet18 循环SNR 全日期闭集
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# 参数设置
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                  # 拼接时每组多少条信号
test_ratio = 0.25        # 测试集比例

# 调用函数
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_all_mix_random(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# ---------- PyTorch & 训练配置 ----------
import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt  # 保留，若需扩展
import math

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据增强 / 信道设置（目前默认关闭）
USE_LOG = True
FS = 20e6
FC = 2.4e9
SNR_DB = -10
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True

# 训练参数
BATCH_SIZE = 256   # 更大 batch 更稳定（你数据很多）
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 1e-4
N_SPLITS = 5
PATIENCE = 8
MIN_DELTA = 0.1   # early stopping 的最小进步（%）

SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

# ---------- 可选：AWGN / Doppler（保留） ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 数据预处理：针对你的交错样本，用 per-sample 标准化，不做 FFT ----------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: np.array [N, L, 2] (I,Q)
    输出 torch.tensor [N, L, 2] (float32)，每个样本做 zero-mean unit-std 标准化（按样本）
    说明：不做 FFT，因为样本不是连续时间序列（已交错）。
    """
    data = data_real_imag.astype(np.float32).copy()
    N, L, C = data.shape
    out = np.empty_like(data, dtype=np.float32)
    for i in range(N):
        iq = data[i]  # (L,2)
        # 可选：先把 (I,Q) 组合成 complex 做 channel 级增强 (保留)
        if add_noise:
            # 以 complex 形式添加噪声
            sigc = iq[...,0] + 1j * iq[...,1]
            sigc = add_complex_awgn(sigc, snr_db)
            if add_doppler:
                sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)
            iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        # per-sample 标准化（对两个通道一起）
        mu = iq.mean(axis=(0,))    # shape (2,)
        sigma = iq.std(axis=(0,))  # shape (2,)
        # 防止除零
        sigma[sigma < 1e-8] = 1.0
        iq_norm = (iq - mu) / sigma
        out[i] = iq_norm

    return torch.tensor(out, dtype=torch.float32)

# ---------- 新的 RF1DCNN ----------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out


class RF1DCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super().__init__()

        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # 240 → 120

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # 120 → 60

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # 60 → 30

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # 30 → 15

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 15, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, 240, 2)
        x = x.permute(0, 2, 1)  # → (B, 2, 240)

        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)

        return self.fc(x)


# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()

# ---------- K-Fold 训练主函数（使用 IQPointCloudCNN） ----------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes, device=DEVICE,
                           batch_size=BATCH_SIZE, epochs=EPOCHS, lr=LR, weight_decay=WEIGHT_DECAY,
                           n_splits=N_SPLITS, patience=PATIENCE, min_delta=MIN_DELTA,
                           script_name="wisig_cross"):
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}dB_fd{fd}_classes_{num_classes}_ResNet"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # test loader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # full dataset for K-Fold
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model = RF1DCNN(num_classes=num_classes, dropout=0.3).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())
            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                  f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                        f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping.")
                    break

            scheduler.step()

        # restore best
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')

        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')

        # test eval
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

        # 保存曲线 & 模型
        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ---------- 运行前的准备（预处理并转换为张量） ----------
# X_train, X_test 目前 shape: [N, 240, 2] (numpy)
# 我们对每个样本做 per-sample 标准化（zero-mean unit-std）
print("Preprocessing (per-sample normalization)...")
# ---------- 转为 torch.tensor 并做 per-sample 标准化 ----------

X_train_torch = preprocess_for_pointcloud_cnn(X_train, add_noise=False)
X_test_torch  = preprocess_for_pointcloud_cnn(X_test, add_noise=False)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch  = torch.tensor(y_test, dtype=torch.long)

num_classes = len(torch.unique(y_train_torch))
print("Prepared tensors:", X_train_torch.shape, X_test_torch.shape, "num_classes=", num_classes)

# ---------- K-Fold 训练 ----------
# train_kfold_pointcloud 内部会把训练集再划分为训练+验证集
# save_folder = train_kfold_pointcloud(
#    X_train_torch, 
#    y_train_torch,
#    X_test_torch, 
#    y_test_torch,
#    num_classes=num_classes
# )

#print("Done. Results in:", save_folder)

##########################################################
#=== 定义一个统一入口函数，用于跑一次 SNR 实验 ===
##########################################################
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE

    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    # 设置当前实验 SNR
    SNR_DB = snr_db
    ADD_NOISE = True   # 打开 AWGN
    
    # --- 重新预处理数据（会调用 add_noise=True） ---
    X_train_torch_snr = preprocess_for_pointcloud_cnn(
        X_train, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )
    X_test_torch_snr = preprocess_for_pointcloud_cnn(
        X_test, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )

    y_train_torch_snr = torch.tensor(y_train, dtype=torch.long)
    y_test_torch_snr  = torch.tensor(y_test,  dtype=torch.long)

    num_classes = len(torch.unique(y_train_torch_snr))

    # --- 调用你的 K-Fold 训练 ---
    save_folder = train_kfold_pointcloud(
        X_train_torch_snr,
        y_train_torch_snr,
        X_test_torch_snr,
        y_test_torch_snr,
        num_classes=num_classes,
        script_name=f"SNR_{snr_db}dB"
    )

    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder


###########################################################
# === 批量执行 SNR Sweep: 20, 10, 0, -10, ... -40 dB ===
###########################################################
snr_list = list(range(20, -41, -5))
# [20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40]

all_results = {}

for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")



In [2]:
# 跨日期 ResNet18 循环SNR 6个设备
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManySig'
dataset_path='../ManySig.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# 参数设置
equalized = 0
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                  # 拼接时每组多少条信号

train_dates = ['2021_03_15']  # 设定训练日期
test_dates  = ['2021_03_01']  # 设定测试日期
# 调用函数
# 调用函数
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_single_date_per_rx_cyclic(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    train_dates=train_dates,
    test_dates=test_dates,
    max_sig=max_sig,        # 每个 RX 最多取多少条信号，可选
    equalized=equalized,        # 是否使用均衡化信号
    block_size=block_size,     # 每个 block 的样本数
    y=y                # 每次循环从 RX 抽取的信号数
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# ---------- PyTorch & 训练配置 ----------
import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt  # 保留，若需扩展
import math

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据增强 / 信道设置（目前默认关闭）
USE_LOG = True
FS = 20e6
FC = 2.4e9
SNR_DB = 20
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True

# 训练参数
BATCH_SIZE = 256   # 更大 batch 更稳定（你数据很多）
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 1e-4
N_SPLITS = 5
PATIENCE = 8
MIN_DELTA = 0.1   # early stopping 的最小进步（%）

SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

# ---------- 可选：AWGN / Doppler（保留） ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 数据预处理：针对你的交错样本，用 per-sample 标准化，不做 FFT ----------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: np.array [N, L, 2] (I,Q)
    输出 torch.tensor [N, L, 2] (float32)，每个样本做 zero-mean unit-std 标准化（按样本）
    说明：不做 FFT，因为样本不是连续时间序列（已交错）。
    """
    data = data_real_imag.astype(np.float32).copy()
    N, L, C = data.shape
    out = np.empty_like(data, dtype=np.float32)
    for i in range(N):
        iq = data[i]  # (L,2)
        # 可选：先把 (I,Q) 组合成 complex 做 channel 级增强 (保留)
        if add_noise:
            # 以 complex 形式添加噪声
            sigc = iq[...,0] + 1j * iq[...,1]
            sigc = add_complex_awgn(sigc, snr_db)
            if add_doppler:
                sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)
            iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        # per-sample 标准化（对两个通道一起）
        mu = iq.mean(axis=(0,))    # shape (2,)
        sigma = iq.std(axis=(0,))  # shape (2,)
        # 防止除零
        sigma[sigma < 1e-8] = 1.0
        iq_norm = (iq - mu) / sigma
        out[i] = iq_norm

    return torch.tensor(out, dtype=torch.float32)

# ---------- 新的 RF1DCNN ----------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out


class RF1DCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super().__init__()

        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # 240 → 120

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # 120 → 60

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # 60 → 30

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # 30 → 15

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 15, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, 240, 2)
        x = x.permute(0, 2, 1)  # → (B, 2, 240)

        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)

        return self.fc(x)


# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()

# ---------- K-Fold 训练主函数（使用 RF1DCNN） ----------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes, device=DEVICE,
                           batch_size=BATCH_SIZE, epochs=EPOCHS, lr=LR, weight_decay=WEIGHT_DECAY,
                           n_splits=N_SPLITS, patience=PATIENCE, min_delta=MIN_DELTA,
                           script_name="wisig_cross"):
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}dB_fd{fd}_classes_{num_classes}_ResNet"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # test loader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # full dataset for K-Fold
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model = RF1DCNN(num_classes=num_classes, dropout=0.3).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())
            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                  f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                        f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping.")
                    break

            scheduler.step()

        # restore best
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')

        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')

        # test eval
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

        # 保存曲线 & 模型
        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ---------- 运行前的准备（预处理并转换为张量） ----------
# X_train, X_test 目前 shape: [N, 240, 2] (numpy)
# 我们对每个样本做 per-sample 标准化（zero-mean unit-std）
print("Preprocessing (per-sample normalization)...")
# ---------- 转为 torch.tensor 并做 per-sample 标准化 ----------

X_train_torch = preprocess_for_pointcloud_cnn(X_train, add_noise=False)
X_test_torch  = preprocess_for_pointcloud_cnn(X_test, add_noise=False)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch  = torch.tensor(y_test, dtype=torch.long)

num_classes = len(torch.unique(y_train_torch))
print("Prepared tensors:", X_train_torch.shape, X_test_torch.shape, "num_classes=", num_classes)

# ---------- K-Fold 训练 ----------
# train_kfold_pointcloud 内部会把训练集再划分为训练+验证集
# save_folder = train_kfold_pointcloud(
#    X_train_torch, 
#    y_train_torch,
#    X_test_torch, 
#    y_test_torch,
#    num_classes=num_classes
# )

#print("Done. Results in:", save_folder)

# ##########################################################
# #=== 定义一个统一入口函数，用于跑一次 SNR 实验 ===
# ##########################################################
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE

    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    # 设置当前实验 SNR
    SNR_DB = snr_db
    ADD_NOISE = True   # 打开 AWGN
    
    # --- 重新预处理数据（会调用 add_noise=True） ---
    X_train_torch_snr = preprocess_for_pointcloud_cnn(
        X_train, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )
    X_test_torch_snr = preprocess_for_pointcloud_cnn(
        X_test, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )

    y_train_torch_snr = torch.tensor(y_train, dtype=torch.long)
    y_test_torch_snr  = torch.tensor(y_test,  dtype=torch.long)

    num_classes = len(torch.unique(y_train_torch_snr))

    # --- 调用你的 K-Fold 训练 ---
    save_folder = train_kfold_pointcloud(
        X_train_torch_snr,
        y_train_torch_snr,
        X_test_torch_snr,
        y_test_torch_snr,
        num_classes=num_classes,
    )

    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder


###########################################################
# === 批量执行 SNR Sweep: 20, 10, 0, -10, ... -40 dB ===
###########################################################
snr_list = list(range(20, -41, -5))
# [20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40]

all_results = {}

for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")

数据集发射机数量： 6 具体为： ['14-10', '14-7', '20-15', '20-19', '6-15', '8-20']
数据集接收机数量： 12 具体为： ['1-1', '1-19', '14-7', '18-2', '19-2', '2-1', '2-19', '20-1', '3-19', '7-14', '7-7', '8-8']
数据集采集天数： 4 具体为： ['2021_03_01', '2021_03_08', '2021_03_15', '2021_03_23']
6 12
X_train shape: (76800, 240, 2)
y_train shape: (76800,)
X_test  shape: (76800, 240, 2)
y_test  shape: (76800,)
Preprocessing (per-sample normalization)...
Prepared tensors: torch.Size([76800, 240, 2]) torch.Size([76800, 240, 2]) num_classes= 6

🚀 Running experiment with SNR = 20 dB

=== Fold 1/5 ===
Epoch 1/200 | TrainAcc=97.12% | ValAcc=99.80% | TrainLoss=0.0825 | ValLoss=0.0061 | AvgGrad=0.0734
Epoch 2/200 | TrainAcc=99.86% | ValAcc=99.84% | TrainLoss=0.0042 | ValLoss=0.0044 | AvgGrad=0.0174
Epoch 3/200 | TrainAcc=99.89% | ValAcc=98.22% | TrainLoss=0.0037 | ValLoss=0.0605 | AvgGrad=0.0159
Epoch 4/200 | TrainAcc=99.81% | ValAcc=99.92% | TrainLoss=0.0054 | ValLoss=0.0031 | AvgGrad=0.0179
Epoch 5/200 | TrainAcc=99.96% | ValAcc=99.82% 

In [ ]:
#!/usr/bin/env python3
# run_date_cross_experiments.py
# 完整：按单日训练/单日测试（4天 -> 16 实验）自动化脚本
# 2025-xx-xx

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.auto import tqdm
from joblib import load
from data_utilities import *   # 你原来的工具函数
import gc
import h5py

# -------------------- 用户可配置项 --------------------
dataset_name = 'ManySig'
dataset_path = '../ManySig.pkl/'   # 请根据本地路径修改
# 若 compact pickle 的加载函数不同，请调整load_compact_pkl_dataset的调用

# 训练/数据参数
EQUALIZED = 0
MAX_SIG = None         # 每个 TX-RX-日期最多使用信号数（None 表示不截断）
BLOCK_SIZE = 240       # 每个 block 的样本数（模型期望长度）
Y = 5                  # 每次从同一个 RX 抽取的信号数
N_SPLITS = 5
BATCH_SIZE = 256
EPOCHS = 100
LR = 1e-3
WEIGHT_DECAY = 1e-4
PATIENCE = 5
MIN_DELTA = 0.1  # early stopping 的最小百分比点数提升

SAVE_ROOT = "./training_results_date_cross"
os.makedirs(SAVE_ROOT, exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("[INFO] Device:", DEVICE)

# -------------------- 加载 compact dataset --------------------
print("[INFO] Loading compact dataset ...")
compact_dataset = load_compact_pkl_dataset(dataset_path, dataset_name)
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
capture_date_list = compact_dataset['capture_date_list']

print("[INFO] tx_list:", tx_list)
print("[INFO] rx_list:", rx_list)
print("[INFO] capture_date_list:", capture_date_list)

# 选择要做单日交叉的前 4 天（如果不足 4 天，则取全部）
num_days = len(capture_date_list)
dates_to_use = capture_date_list[:num_days]
print(f"[INFO] Using {len(dates_to_use)} dates for cross experiments:", dates_to_use)

# -------------------- 数据预处理函数（单日训练 / 单日测试） --------------------
def preprocess_dataset_cross_IQ_blocks_single_date_per_rx_cyclic(compact_dataset, tx_list, train_dates, test_dates,
                                                                 max_sig=None, equalized=0, block_size=240, y=5):
    """
    从 compact_dataset 中按指定 train_dates 与 test_dates 分别提取样本。
    同 RX 周期性抽样拼接成 blocks（block_size），然后转置得到每个样本长度为 block_size，通道为 I/Q。
    返回：X_train (N_train, block_size, 2), y_train (N_train,), X_test, y_test
    """
    import numpy as np

    def extract_samples_for_dates(dates):
        X = []
        y_labels = []
        # tx_list 的顺序决定类标签
        for tx_idx, tx in enumerate(tx_list):
            tx_i = compact_dataset['tx_list'].index(tx)
            # equalized 找 index
            if equalized in compact_dataset['equalized_list']:
                eq_i = compact_dataset['equalized_list'].index(equalized)
            else:
                eq_i = 0
            for date in dates:
                if date not in compact_dataset['capture_date_list']:
                    # 跳过没有的日期
                    continue
                date_i = compact_dataset['capture_date_list'].index(date)

                # 收集每个 RX 的信号列表（并打乱顺序）
                rx_signals = []
                for rx_i in range(len(compact_dataset['rx_list'])):
                    # 可能数据为空或形状不同，需安全取
                    sig_data = compact_dataset['data'][tx_i][rx_i][date_i][eq_i]
                    if max_sig is not None:
                        sig_data = sig_data[:max_sig]
                    # 将 array 转为 list，便于 pop
                    sig_list = list(sig_data.copy())
                    # 随机化顺序
                    np.random.shuffle(sig_list)
                    rx_signals.append(sig_list)

                num_rx = len(rx_signals)
                rx_pointer = 0
                accum_block = []

                # 循环直到所有 rx 的信号耗尽
                while any(len(sig_list) > 0 for sig_list in rx_signals):
                    rx_idx = rx_pointer % num_rx
                    sig_list = rx_signals[rx_idx]
                    if len(sig_list) > 0:
                        take_n = min(y, len(sig_list))
                        # 顺序取出 take_n 个
                        sampled = [sig_list.pop(0) for _ in range(take_n)]
                        accum_block.extend(sampled)
                    rx_pointer += 1

                    # 当累积够一个 block 时，切分并把每个采样点当成一个样本（transpose）
                    while len(accum_block) >= block_size:
                        block_chunk = accum_block[:block_size]
                        accum_block = accum_block[block_size:]
                        block_array = np.array(block_chunk)  # (block_size, sample_len, 2) — 注意 sample_len 是原单个信号长度
                        # 这里原始信号长度（例如 256）可能在第 1 维，因此需要转置为 (sample_len, block_size, 2)
                        # 原始信号通常是 (L,2) 单信号。 block_array -> (block_size, L, 2)
                        # 我们要把每个采样点（0..L-1）当成样本：shape -> (L, block_size, 2) -> 然后把每个 j 当作一个样本
                        block_transposed = block_array.transpose(1, 0, 2)  # (L, block_size, 2)
                        # 对于每个采样点 j, 添加样本长度为 block_size 的样本，通道 I/Q
                        for j in range(block_transposed.shape[0]):
                            X.append(block_transposed[j])
                            y_labels.append(tx_idx)
                # 丢弃剩余的 accum_block（不足 block_size）
                accum_block = []

        if len(X) == 0:
            return np.zeros((0, block_size, 2), dtype=np.float32), np.zeros((0,), dtype=np.int64)
        return np.array(X, dtype=np.float32), np.array(y_labels, dtype=np.int64)

    X_train, y_train = extract_samples_for_dates(train_dates)
    X_test, y_test = extract_samples_for_dates(test_dates)
    return X_train, y_train, X_test, y_test

# -------------------- 模型定义：RF1DResidual（与你之前的 RF1DCNN 类似） --------------------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(identity)
        out += identity
        out = self.relu(out)
        return out

class RF1DResNet(nn.Module):
    def __init__(self, num_classes, dropout=0.3, input_length=BLOCK_SIZE):
        super().__init__()
        # 输入是 (B, L, 2) -> 转为 (B, 2, L)
        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # L -> L/2

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # L/4

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # L/8

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # L/16

        # 计算 flatten 大小（向下取整）
        L_after = input_length
        for _ in range(4):
            L_after = (L_after + 1) // 2  # approximate for pool1d/2
        self.flatten_dim = 256 * L_after

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, L, 2)
        x = x.permute(0, 2, 1)  # -> (B, 2, L)
        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)
        return self.fc(x)

# -------------------- 训练与评估辅助函数 --------------------
def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            total_norm += (p.grad.data.norm(2).item()) ** 2
    return total_norm ** 0.5

def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total > 0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix_save(cm, classes, save_path, title='Confusion Matrix'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(save_path)
    plt.close()

def plot_curves_save(train_losses, val_losses, train_acc, val_acc, save_prefix):
    # loss curve
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.savefig(save_prefix + '_loss.png'); plt.close()
    # acc curve
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.grid(True)
    plt.savefig(save_prefix + '_acc.png'); plt.close()

# -------------------- K-Fold 训练函数（并保存每 fold 的所有 artefacts） --------------------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes,
                           device=DEVICE, batch_size=BATCH_SIZE, epochs=EPOCHS,
                           lr=LR, weight_decay=WEIGHT_DECAY, n_splits=N_SPLITS,
                           patience=PATIENCE, min_delta=MIN_DELTA, save_folder=None,
                           script_name="date_cross"):
    """
    X_train: torch.tensor or numpy array (N, L, 2)
    y_train: torch.tensor or numpy (N,)
    X_test, y_test: same form
    save_folder: where to save per-experiment outputs
    """
    if save_folder is None:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        save_folder = os.path.join(SAVE_ROOT, f"{timestamp}_{script_name}")
        os.makedirs(save_folder, exist_ok=True)

    results_file = os.path.join(save_folder, "results.txt")

    # 将输入转换为 TensorDataset（如果还不是 tensor）
    if not torch.is_tensor(X_train):
        X_train_t = torch.tensor(X_train, dtype=torch.float32)
    else:
        X_train_t = X_train
    if not torch.is_tensor(y_train):
        y_train_t = torch.tensor(y_train, dtype=torch.long)
    else:
        y_train_t = y_train
    if not torch.is_tensor(X_test):
        X_test_t = torch.tensor(X_test, dtype=torch.float32)
    else:
        X_test_t = X_test
    if not torch.is_tensor(y_test):
        y_test_t = torch.tensor(y_test, dtype=torch.long)
    else:
        y_test_t = y_test

    test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size, shuffle=False)

    # 写入实验参数头
    with open(results_file, 'a') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Device: {DEVICE}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"K-Fold: {n_splits}, Patience: {patience}\n")
        f.write("============================\n\n")

    full_dataset = TensorDataset(X_train_t, y_train_t)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    # K-Fold loop
    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n[Experiment] Fold {fold+1}/{n_splits}")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False)

        model = RF1DResNet(num_classes=num_classes, input_length=BLOCK_SIZE).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses = [], []
        train_accs, val_accs = [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)

            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())

            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)

            # 打印并写入 results_file
            line = f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}"
            print(line)
            with open(results_file, 'a') as f:
                f.write(line + "\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("[INFO] Early stopping triggered.")
                    break

            scheduler.step()

        # restore best weights for this fold
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵和曲线 & 模型
        train_acc_fold, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix_save(train_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'train_cm_fold{fold+1}.png'),
                                   title=f'Train CM Fold {fold+1}')

        val_acc_fold, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix_save(val_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'val_cm_fold{fold+1}_val.png'),
                                   title=f'Val CM Fold {fold+1}')

        test_acc_fold, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix_save(test_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'test_cm_fold{fold+1}.png'),
                                   title=f'Test CM Fold {fold+1}')

        # curve plots per fold
        plot_curves_save(train_losses, val_losses, train_accs, val_accs, save_prefix=os.path.join(save_folder, f'fold{fold+1}'))

        # save model
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        # append fold metrics
        val_scores.append(val_acc); test_scores.append(test_acc_fold)

        print(f"🔥 Fold {fold+1} - TEST Accuracy: {test_acc_fold:.4f}")
        
        # 写 fold summary
        with open(results_file, 'a') as f:
            f.write(f"\n=== Fold {fold+1} Summary ===\n")
            f.write(f"TrainAcc (last) = {train_acc:.2f}%\n")
            f.write(f"ValAcc (best) = {best_val:.2f}%\n")
            f.write(f"TestAcc = {test_acc_fold:.2f}%\n")
            f.write("===========================\n\n")

        # 清理显存
        del model; gc.collect(); torch.cuda.empty_cache()

    # overall summary
    with open(results_file, 'a') as f:
        f.write("\n=== Overall Summary ===\n")
        f.write(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\n")
        f.write(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")
        f.write("=======================\n")
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")

    return os.path.abspath(save_folder)

# -------------------- 主控制流程：16 次实验循环 --------------------
def main_all_date_experiments():
    # take first num_days dates
    dates = dates_to_use
    all_experiment_folders = []
    for train_date in dates:
        for test_date in dates:
            # 构造 folder 名称包含训练/测试日期
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            folder_name = f"{timestamp}_train{train_date}_test{test_date}"
            save_folder = os.path.join(SAVE_ROOT, folder_name)
            os.makedirs(save_folder, exist_ok=True)
            results_file = os.path.join(save_folder, "results.txt")

            with open(results_file, "a") as f:
                f.write(f"Experiment Timestamp: {timestamp}\n")
                f.write(f"Train Date: {train_date}\n")
                f.write(f"Test Date: {test_date}\n")
                f.write("========================\n")

            print(f"\n\n===== Experiment: train={train_date}  test={test_date}  => folder: {save_folder} =====\n")
            # 预处理：分别按 train_date 和 test_date 提取数据
            X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_single_date_per_rx_cyclic(
                compact_dataset=compact_dataset,
                tx_list=tx_list,
                train_dates=[train_date],
                test_dates=[test_date],
                max_sig=MAX_SIG,
                equalized=EQUALIZED,
                block_size=BLOCK_SIZE,
                y=Y
            )

            # 如果数据为空，跳过
            if X_train.shape[0] == 0 or X_test.shape[0] == 0:
                print(f"[WARN] Empty train/test for train={train_date} test={test_date}, skipping.")
                with open(results_file, "a") as f:
                    f.write("SKIPPED: empty train/test\n")
                continue

            # 把数据标准化为 per-sample zero-mean unit-std（按 sample 的两个通道一起）
            def per_sample_normalize(X):
                Xn = X.astype(np.float32).copy()
                N = Xn.shape[0]
                for i in range(N):
                    mu = Xn[i].mean(axis=0)   # (2,)
                    sigma = Xn[i].std(axis=0)
                    sigma[sigma < 1e-8] = 1.0
                    Xn[i] = (Xn[i] - mu) / sigma
                return Xn
            X_train_n = per_sample_normalize(X_train)
            X_test_n = per_sample_normalize(X_test)

            # 转为 torch tensors
            X_train_t = torch.tensor(X_train_n, dtype=torch.float32)
            y_train_t = torch.tensor(y_train, dtype=torch.long)
            X_test_t = torch.tensor(X_test_n, dtype=torch.float32)
            y_test_t = torch.tensor(y_test, dtype=torch.long)

            num_classes = len(np.unique(y_train))
            # 写入 result 文件简要信息
            with open(results_file, "a") as f:
                f.write(f"Num train samples: {len(X_train_t)}\n")
                f.write(f"Num test samples: {len(X_test_t)}\n")
                f.write(f"Num classes (from train): {num_classes}\n")
                f.write("========================\n")

            # 训练并保存
            exp_folder = train_kfold_pointcloud(X_train_t, y_train_t, X_test_t, y_test_t,
                                               num_classes=num_classes, save_folder=save_folder,
                                               script_name=f"train{train_date}_test{test_date}")
            all_experiment_folders.append(exp_folder)

    # summary of all experiments
    print("\nAll experiments done. Results folders:")
    for p in all_experiment_folders:
        print(p)
    return all_experiment_folders

if __name__ == "__main__":
    folders = main_all_date_experiments()
    print("\nFinished. Saved experiment folders:")
    for fd in folders:
        print(fd)


In [ ]:
#!/usr/bin/env python3
# run_date_cross_experiments.py
# ManyTx完整：按单日训练/单日测试（4天 -> 16 实验）自动化脚本
# 2025-xx-xx

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.auto import tqdm
from joblib import load
from data_utilities import *   # 你原来的工具函数
import gc
import h5py

# -------------------- 用户可配置项 --------------------
dataset_name = 'ManyTx'
dataset_path = '../ManyTx.pkl/'   # 请根据本地路径修改
# 若 compact pickle 的加载函数不同，请调整load_compact_pkl_dataset的调用

# 训练/数据参数
EQUALIZED = 0
MAX_SIG = None         # 每个 TX-RX-日期最多使用信号数（None 表示不截断）
BLOCK_SIZE = 240       # 每个 block 的样本数（模型期望长度）
Y = 5                  # 每次从同一个 RX 抽取的信号数
N_SPLITS = 5
BATCH_SIZE = 256
EPOCHS = 100
LR = 1e-3
WEIGHT_DECAY = 1e-4
PATIENCE = 5
MIN_DELTA = 0.1  # early stopping 的最小百分比点数提升

SAVE_ROOT = "./training_results_date_cross"
os.makedirs(SAVE_ROOT, exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("[INFO] Device:", DEVICE)

# -------------------- 加载 compact dataset --------------------
print("[INFO] Loading compact dataset ...")
compact_dataset = load_compact_pkl_dataset(dataset_path, dataset_name)
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
capture_date_list = compact_dataset['capture_date_list']

print("[INFO] tx_list:", tx_list)
print("[INFO] rx_list:", rx_list)
print("[INFO] capture_date_list:", capture_date_list)

# 选择要做单日交叉的前 4 天（如果不足 4 天，则取全部）
num_days = len(capture_date_list)
dates_to_use = capture_date_list[:num_days]
print(f"[INFO] Using {len(dates_to_use)} dates for cross experiments:", dates_to_use)

# -------------------- 数据预处理函数（单日训练 / 单日测试） --------------------
def preprocess_dataset_cross_IQ_blocks_single_date_per_rx_cyclic(compact_dataset, tx_list, train_dates, test_dates,
                                                                 max_sig=None, equalized=0, block_size=240, y=5):
    """
    从 compact_dataset 中按指定 train_dates 与 test_dates 分别提取样本。
    同 RX 周期性抽样拼接成 blocks（block_size），然后转置得到每个样本长度为 block_size，通道为 I/Q。
    返回：X_train (N_train, block_size, 2), y_train (N_train,), X_test, y_test
    """
    import numpy as np

    def extract_samples_for_dates(dates):
        X = []
        y_labels = []
        # tx_list 的顺序决定类标签
        for tx_idx, tx in enumerate(tx_list):
            tx_i = compact_dataset['tx_list'].index(tx)
            # equalized 找 index
            if equalized in compact_dataset['equalized_list']:
                eq_i = compact_dataset['equalized_list'].index(equalized)
            else:
                eq_i = 0
            for date in dates:
                if date not in compact_dataset['capture_date_list']:
                    # 跳过没有的日期
                    continue
                date_i = compact_dataset['capture_date_list'].index(date)

                # 收集每个 RX 的信号列表（并打乱顺序）
                rx_signals = []
                for rx_i in range(len(compact_dataset['rx_list'])):
                    # 可能数据为空或形状不同，需安全取
                    sig_data = compact_dataset['data'][tx_i][rx_i][date_i][eq_i]
                    if max_sig is not None:
                        sig_data = sig_data[:max_sig]
                    # 将 array 转为 list，便于 pop
                    sig_list = list(sig_data.copy())
                    # 随机化顺序
                    np.random.shuffle(sig_list)
                    rx_signals.append(sig_list)

                num_rx = len(rx_signals)
                rx_pointer = 0
                accum_block = []

                # 循环直到所有 rx 的信号耗尽
                while any(len(sig_list) > 0 for sig_list in rx_signals):
                    rx_idx = rx_pointer % num_rx
                    sig_list = rx_signals[rx_idx]
                    if len(sig_list) > 0:
                        take_n = min(y, len(sig_list))
                        # 顺序取出 take_n 个
                        sampled = [sig_list.pop(0) for _ in range(take_n)]
                        accum_block.extend(sampled)
                    rx_pointer += 1

                    # 当累积够一个 block 时，切分并把每个采样点当成一个样本（transpose）
                    while len(accum_block) >= block_size:
                        block_chunk = accum_block[:block_size]
                        accum_block = accum_block[block_size:]
                        block_array = np.array(block_chunk)  # (block_size, sample_len, 2) — 注意 sample_len 是原单个信号长度
                        # 这里原始信号长度（例如 256）可能在第 1 维，因此需要转置为 (sample_len, block_size, 2)
                        # 原始信号通常是 (L,2) 单信号。 block_array -> (block_size, L, 2)
                        # 我们要把每个采样点（0..L-1）当成样本：shape -> (L, block_size, 2) -> 然后把每个 j 当作一个样本
                        block_transposed = block_array.transpose(1, 0, 2)  # (L, block_size, 2)
                        # 对于每个采样点 j, 添加样本长度为 block_size 的样本，通道 I/Q
                        for j in range(block_transposed.shape[0]):
                            X.append(block_transposed[j])
                            y_labels.append(tx_idx)
                # 丢弃剩余的 accum_block（不足 block_size）
                accum_block = []

        if len(X) == 0:
            return np.zeros((0, block_size, 2), dtype=np.float32), np.zeros((0,), dtype=np.int64)
        return np.array(X, dtype=np.float32), np.array(y_labels, dtype=np.int64)

    X_train, y_train = extract_samples_for_dates(train_dates)
    X_test, y_test = extract_samples_for_dates(test_dates)
    return X_train, y_train, X_test, y_test

# -------------------- 模型定义：RF1DResidual（与你之前的 RF1DCNN 类似） --------------------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(identity)
        out += identity
        out = self.relu(out)
        return out

class RF1DResNet(nn.Module):
    def __init__(self, num_classes, dropout=0.3, input_length=BLOCK_SIZE):
        super().__init__()
        # 输入是 (B, L, 2) -> 转为 (B, 2, L)
        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # L -> L/2

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # L/4

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # L/8

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # L/16

        # 计算 flatten 大小（向下取整）
        L_after = input_length
        for _ in range(4):
            L_after = (L_after + 1) // 2  # approximate for pool1d/2
        self.flatten_dim = 256 * L_after

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, L, 2)
        x = x.permute(0, 2, 1)  # -> (B, 2, L)
        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)
        return self.fc(x)

# -------------------- 训练与评估辅助函数 --------------------
def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            total_norm += (p.grad.data.norm(2).item()) ** 2
    return total_norm ** 0.5

def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total > 0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix_save(cm, classes, save_path, title='Confusion Matrix'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(save_path)
    plt.close()

def plot_curves_save(train_losses, val_losses, train_acc, val_acc, save_prefix):
    # loss curve
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.savefig(save_prefix + '_loss.png'); plt.close()
    # acc curve
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.grid(True)
    plt.savefig(save_prefix + '_acc.png'); plt.close()

# -------------------- K-Fold 训练函数（并保存每 fold 的所有 artefacts） --------------------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes,
                           device=DEVICE, batch_size=BATCH_SIZE, epochs=EPOCHS,
                           lr=LR, weight_decay=WEIGHT_DECAY, n_splits=N_SPLITS,
                           patience=PATIENCE, min_delta=MIN_DELTA, save_folder=None,
                           script_name="date_cross"):
    """
    X_train: torch.tensor or numpy array (N, L, 2)
    y_train: torch.tensor or numpy (N,)
    X_test, y_test: same form
    save_folder: where to save per-experiment outputs
    """
    if save_folder is None:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        save_folder = os.path.join(SAVE_ROOT, f"{timestamp}_{script_name}")
        os.makedirs(save_folder, exist_ok=True)

    results_file = os.path.join(save_folder, "results.txt")

    # 将输入转换为 TensorDataset（如果还不是 tensor）
    if not torch.is_tensor(X_train):
        X_train_t = torch.tensor(X_train, dtype=torch.float32)
    else:
        X_train_t = X_train
    if not torch.is_tensor(y_train):
        y_train_t = torch.tensor(y_train, dtype=torch.long)
    else:
        y_train_t = y_train
    if not torch.is_tensor(X_test):
        X_test_t = torch.tensor(X_test, dtype=torch.float32)
    else:
        X_test_t = X_test
    if not torch.is_tensor(y_test):
        y_test_t = torch.tensor(y_test, dtype=torch.long)
    else:
        y_test_t = y_test

    test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size, shuffle=False)

    # 写入实验参数头
    with open(results_file, 'a') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Device: {DEVICE}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"K-Fold: {n_splits}, Patience: {patience}\n")
        f.write("============================\n\n")

    full_dataset = TensorDataset(X_train_t, y_train_t)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    # K-Fold loop
    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n[Experiment] Fold {fold+1}/{n_splits}")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False)

        model = RF1DResNet(num_classes=num_classes, input_length=BLOCK_SIZE).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses = [], []
        train_accs, val_accs = [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)

            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())

            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)

            # 打印并写入 results_file
            line = f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}"
            print(line)
            with open(results_file, 'a') as f:
                f.write(line + "\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("[INFO] Early stopping triggered.")
                    break

            scheduler.step()

        # restore best weights for this fold
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵和曲线 & 模型
        train_acc_fold, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix_save(train_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'train_cm_fold{fold+1}.png'),
                                   title=f'Train CM Fold {fold+1}')

        val_acc_fold, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix_save(val_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'val_cm_fold{fold+1}_val.png'),
                                   title=f'Val CM Fold {fold+1}')

        test_acc_fold, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix_save(test_cm, classes=list(range(num_classes)),
                                   save_path=os.path.join(save_folder, f'test_cm_fold{fold+1}.png'),
                                   title=f'Test CM Fold {fold+1}')

        # curve plots per fold
        plot_curves_save(train_losses, val_losses, train_accs, val_accs, save_prefix=os.path.join(save_folder, f'fold{fold+1}'))

        # save model
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        # append fold metrics
        val_scores.append(val_acc); test_scores.append(test_acc_fold)

        print(f"🔥 Fold {fold+1} - TEST Accuracy: {test_acc_fold:.4f}")
        
        # 写 fold summary
        with open(results_file, 'a') as f:
            f.write(f"\n=== Fold {fold+1} Summary ===\n")
            f.write(f"TrainAcc (last) = {train_acc:.2f}%\n")
            f.write(f"ValAcc (best) = {best_val:.2f}%\n")
            f.write(f"TestAcc = {test_acc_fold:.2f}%\n")
            f.write("===========================\n\n")

        # 清理显存
        del model; gc.collect(); torch.cuda.empty_cache()

    # overall summary
    with open(results_file, 'a') as f:
        f.write("\n=== Overall Summary ===\n")
        f.write(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\n")
        f.write(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")
        f.write("=======================\n")
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")

    return os.path.abspath(save_folder)

# -------------------- 主控制流程：16 次实验循环 --------------------
def main_all_date_experiments():
    # take first num_days dates
    dates = dates_to_use
    all_experiment_folders = []
    for train_date in dates:
        for test_date in dates:
            # 构造 folder 名称包含训练/测试日期
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            folder_name = f"{timestamp}_train{train_date}_test{test_date}_ManyTx"
            save_folder = os.path.join(SAVE_ROOT, folder_name)
            os.makedirs(save_folder, exist_ok=True)
            results_file = os.path.join(save_folder, "results.txt")

            with open(results_file, "a") as f:
                f.write(f"Experiment Timestamp: {timestamp}\n")
                f.write(f"Train Date: {train_date}\n")
                f.write(f"Test Date: {test_date}\n")
                f.write("========================\n")

            print(f"\n\n===== Experiment: train={train_date}  test={test_date}  => folder: {save_folder} =====\n")
            # 预处理：分别按 train_date 和 test_date 提取数据
            X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_single_date_per_rx_cyclic(
                compact_dataset=compact_dataset,
                tx_list=tx_list,
                train_dates=[train_date],
                test_dates=[test_date],
                max_sig=MAX_SIG,
                equalized=EQUALIZED,
                block_size=BLOCK_SIZE,
                y=Y
            )

            # 如果数据为空，跳过
            if X_train.shape[0] == 0 or X_test.shape[0] == 0:
                print(f"[WARN] Empty train/test for train={train_date} test={test_date}, skipping.")
                with open(results_file, "a") as f:
                    f.write("SKIPPED: empty train/test\n")
                continue

            # 把数据标准化为 per-sample zero-mean unit-std（按 sample 的两个通道一起）
            def per_sample_normalize(X):
                Xn = X.astype(np.float32).copy()
                N = Xn.shape[0]
                for i in range(N):
                    mu = Xn[i].mean(axis=0)   # (2,)
                    sigma = Xn[i].std(axis=0)
                    sigma[sigma < 1e-8] = 1.0
                    Xn[i] = (Xn[i] - mu) / sigma
                return Xn
            X_train_n = per_sample_normalize(X_train)
            X_test_n = per_sample_normalize(X_test)

            # 转为 torch tensors
            X_train_t = torch.tensor(X_train_n, dtype=torch.float32)
            y_train_t = torch.tensor(y_train, dtype=torch.long)
            X_test_t = torch.tensor(X_test_n, dtype=torch.float32)
            y_test_t = torch.tensor(y_test, dtype=torch.long)

            num_classes = len(np.unique(y_train))
            # 写入 result 文件简要信息
            with open(results_file, "a") as f:
                f.write(f"Num train samples: {len(X_train_t)}\n")
                f.write(f"Num test samples: {len(X_test_t)}\n")
                f.write(f"Num classes (from train): {num_classes}\n")
                f.write("========================\n")

            # 训练并保存
            exp_folder = train_kfold_pointcloud(X_train_t, y_train_t, X_test_t, y_test_t,
                                               num_classes=num_classes, save_folder=save_folder,
                                               script_name=f"train{train_date}_test{test_date}")
            all_experiment_folders.append(exp_folder)

    # summary of all experiments
    print("\nAll experiments done. Results folders:")
    for p in all_experiment_folders:
        print(p)
    return all_experiment_folders

if __name__ == "__main__":
    folders = main_all_date_experiments()
    print("\nFinished. Saved experiment folders:")
    for fd in folders:
        print(fd)
